# Table of contents:
 * [Part 1](#Introduction-to-Machine-Learning---Part-1)
 

# Introduction to Machine Learning - Part 1

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random
import pandas as pd
import itertools
import importlib
import mlintro_min as mli

In [ ]:
%matplotlib inline

# Simple regression (OLS) model

Range of cannon angle psi to investigate

In [ ]:
psi_min=20
psi_max=60

Read pre-generated source of data

In [ ]:
ref_df = pd.read_csv('datasets/offline/refdata_100K.csv')
# Get a very small subset of inter-spread data for plotting
ref_df_light = mli.get_ref_light(ref_df, psi_min=psi_min, psi_max=psi_max)

Generate a single dataset of 50 observations

In [ ]:
np.random.seed(42)
ds = mli.get_datasets(ref_df, n_datasets=1, sample_size=50, 
                      psi_min=psi_min, psi_max=psi_max)[0]

##### Explore dataset

We are only interested in experimental measures of angle and range, `exp_angle` and `exp_range` respectively

In [ ]:
ds

### Exercise 
Plot the (experimentally) measured range `exp_range` as a function of the (experimentally) measured angle `exp_angle`, once with matplotlib `plt.scatter()` function, and once with seaborn `sns.scatterplot()` function.  

* HINT 1: look at the functions help to know which function arguments to use
* HINT 2: you can call these functions in two ways:
  * either extract the columns you want to plot and give them as parameters `x` and `y`, OR
  * just provide the column names as arguments for `x` and `y`,  and provide the dataframe name as argument to the parameter `data`. This approach is preferred.
* HINT 3: if using the first method mentioned in HINT 2, remember you can extract a column from a dataframe either as a single column `pandas` object called a `Series` with `[ ]`, or as a `DataFrame` object with `[[ ]]`

In [ ]:
plt.figure(figsize=(8, 4))
#Write plot commands here
plt.ylim([50, 80]);

## Part 1: model selection with train / test split

We will compare three models of different complexity (different number of parameters):
   * Standard linear model (2 parameters: $\beta_0$, $\beta_\psi$)
   * Linear model with the feature squared (3 parameters: $\beta_0$, $\beta_\psi$, $\beta_{\psi^2}$)
   * Linear model with up to $5^{th}$ power of feature (6 parameters: $\beta_0$, $\beta_\psi$, $\beta_{\psi^2}$, $\beta_{\psi^3}$, $\beta_{\psi^4}$, $\beta_{\psi^5}$)

In [ ]:
from sklearn.model_selection import train_test_split

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

Let's divide our dataset into training and test set !

In [ ]:
X = ds[['exp_angle']]
y = ds[['exp_range']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
print(f"Number of observations: {len(X_train)} for training and {len(X_test)} for testing")

In [ ]:
# We simply used tuple unpacking
a, b, c = 1, 13, 42
print(f'a={a}, b={b}, c={c}')

In [ ]:
y_test

### Linear (OLS) model with single feature

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

##### Let's train the model

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

Create new estimator object (each model is an object in `sklearn`)

In [ ]:
lm = LinearRegression()

In [ ]:
lm.

In [ ]:
lm.fit(X_train, y_train)

In [ ]:
lm?

In [ ]:
print(f'beta_psi: {lm.coef_}, beta_0: {lm.intercept_}')

##### Predict, and compute performance scores

In [ ]:
lm.predict([[30], [45], [60]])

The training score can provide an idea of best possible performance

In [ ]:
# We score the predictions made on the same data we trained on
y_pred = lm.predict(X_train)

R2_train = r2_score(y_train, y_pred)
MSE_train = mean_squared_error(y_train, y_pred)
print(f'lm training performance is R2: {R2_train:0.2f} and MSE: {MSE_train:0.2f}')

`R2` can be obtained directly from the model / estimator object as it is the default for `LinearRegression`

In [ ]:
lm.score(X_train, y_train)

Let's save all our models scores to compare models later

In [ ]:
train_test_results = []
train_test_results.append({'model': 'lm', 'stage': 'train', 
                           'scorer': 'r2', 'val': R2_train})
train_test_results.append({'model': 'lm', 'stage': 'train', 
                           'scorer': 'MSE', 'val': -MSE_train})

##### Let's test on unseen data and get testing score

See possible scores: https://scikit-learn.org/stable/modules/model_evaluation.html

The testing score can provide an idea of performance for unseen data

### Exercise
Let's use the fitted model to see its performance on unseen data. As above, use the `lm.predict()`, `r2_score()` and `mean_squared_error()` functions, but this time to get the scores on the unseen data `X_test`. The predictions of `X_test` have to be compared with the ground truth data `y_test`.
* HINT 1: copy the relevant code, and replace the training data with the testing data where required
* HINT 2: you should find as performance R2: 0.26 and MSE: 33.05

In [ ]:
# We score the predictions made on unseen data

Again let's save our scores for comparison later

In [ ]:
train_test_results.append({'model': 'lm', 'stage': 'test', 
                           'scorer': 'r2', 'val': R2_test})
train_test_results.append({'model': 'lm', 'stage': 'test', 
                           'scorer': 'MSE', 'val': -MSE_test})

### Polynomial degree 2 model (polynomial use feature powers as additional features)

##### Train and get training score of degree 2 polynomial

The polynomial terms are created and then added to a standard `LinearRegression`

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html

In [ ]:
poly_transformer = PolynomialFeatures(degree=2)
lm_deg2 = LinearRegression()

A polynomial model is fitted by first transforming the features and then training a `LinearRegression` on the transformed features

In [ ]:
X_train_deg2 = poly_transformer.fit_transform(X_train)

In [ ]:
X_train

In [ ]:
X_train_deg2

In [ ]:
lm_deg2.fit(X_train_deg2, y_train)

In [ ]:
lm_deg2.coef_

Let's compute training score  
**Warning**: `LinearRegression` needs to be applied to the *transformed* (polynomial) features

In [ ]:
y_pred = lm_deg2.predict(X_train_deg2)

R2_train = r2_score(y_train, y_pred)
MSE_train = mean_squared_error(y_train, y_pred)
print(f'poly deg2 training performance is R2: {R2_train:0.2f} and MSE: {MSE_train:0.2f}')

In [ ]:
train_test_results.append({'model': 'lm_deg2', 'stage': 'train', 
                           'scorer': 'r2', 'val': R2_train})
train_test_results.append({'model': 'lm_deg2', 'stage': 'train', 
                           'scorer': 'MSE', 'val': -MSE_train})

##### Let's test on unseen data and get testing score

Again, should not forget to transform the test data to get polynomial features

In [ ]:
X_test_deg2 = poly_transformer.fit_transform(X_test) 

y_pred = lm_deg2.predict(X_test_deg2)

R2_test = r2_score(y_test, y_pred)
MSE_test = mean_squared_error(y_test, y_pred)
print(f'poly deg2 testing performance is R2: {R2_test:0.2f} and MSE: {MSE_test:0.2f}')

In [ ]:
train_test_results.append({'model': 'lm_deg2', 'stage': 'test', 
                           'scorer': 'r2', 'val': R2_test})
train_test_results.append({'model': 'lm_deg2', 'stage': 'test', 
                           'scorer': 'MSE', 'val': -MSE_test})

### Polynomial degree 5 model, introducing pipeline object

It is common to use preprocessing steps such as features transformation. To avoid repeating these processing steps (e.g. when testing the model on new data) the `Pipeline` object is very useful. It builds a workflow which can be called with a single command. 

##### Train and get training score of degree 5 polynomial with *pipeline*

In [ ]:
from sklearn.pipeline import Pipeline

https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

The creation of the polynomial terms are embed in the `Pipeline` object which represents our new estimator (i.e. the one to use with `fit`, `predict`, etc.)

In [ ]:
lm_deg5 = Pipeline([('poly_transformer', PolynomialFeatures(degree=5)),
                    ('lm', LinearRegression())])

### Exercise
Train and test your "degree 5"-polynomial model, and get both the training and testing scores (R2 and MSE).  
1. Fit  your pipeline model on `X_train` and `y_train` with the `lm_deg5.fit()` method (you only need to call this method because the pipeline object will take care of calling the required `transform` and `fit` methods of all the pipeline objects)
2. Examine your fitted linear model coefficients. Because your model is a pipeline object including `poly_transformer` and `lm`, you have to look at the specific `lm` object accessible with `lm_deg5['lm']` 
3. Compute the R2 and MSE training score by comparing predictions on `X_train` (you can name them `y_pred`) with the ground truth `y_train`, and save them in the variables `R2_train` and `MSE_train` respectively
4. Run the cell further down below to append the training scores to the `train_test_results` variable
5. Using the fitted model on step 1, now compute the R2 and MSE testing scores by comparing predictions on `X_test` with `y_test`, and save them in the variables `R2_test` and `MSE_test` respectively
6. Run the cell further down below to append the test scores to the `train_test_results` variable

HINT: Copy previous code and adapt it as needed

In [ ]:
# For Step 4
train_test_results.append({'model': 'lm_deg5', 'stage': 'train', 
                           'scorer': 'r2', 'val': R2_train})
train_test_results.append({'model': 'lm_deg5', 'stage': 'train', 
                           'scorer': 'MSE', 'val': -MSE_train})

In [ ]:
# For Step 6
train_test_results.append({'model': 'lm_deg5', 'stage': 'test', 
                           'scorer': 'r2', 'val': R2_test})
train_test_results.append({'model': 'lm_deg5', 'stage': 'test', 
                           'scorer': 'MSE', 'val': -MSE_test})

##### Plot all the results

In [ ]:
# The list of dictionaries is transformed into a Pandas dataframe
train_test_results_df = pd.DataFrame(train_test_results)
# We extract the r2 score results
r2_results = train_test_results_df.loc[train_test_results_df['scorer'] == 'r2']
# We extract the MSE score results
MSE_results = train_test_results_df.loc[train_test_results_df['scorer'] == 'MSE']
# We call the seaborn `catplot` function which can plot 4 level of informations:
# x [model: lm, lm_deg2, ...], y [score value], colored by `hue` [training or testing score], 
# and producing separate plots for each `col` [score metric: r2 or MSE]
with sns.plotting_context("notebook", font_scale=1.2):
    g = sns.catplot(x="model", y="val", hue="stage", col="scorer", 
                    data=train_test_results_df, kind="bar", sharey=False)

For many other regression models implemented in sklearn, cf https://scikit-learn.org/stable/supervised_learning.html#supervised-learning 

## Part 2: Model selection with cross-validation

https://scikit-learn.org/stable/modules/cross_validation.html

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
KFold?

Let's see what K fold is actually doing on a 10-observation dataset

In [ ]:
X_example = pd.DataFrame({'obs': 
                          np.random.randint(0, 1000, 10)})
X_example

In [ ]:
kf = KFold(n_splits=5)
for (ix_train, ix_test) in kf.split(X_example):
    print("train ix:", ix_train, "testix:", ix_test)

In [ ]:
kfs = KFold(n_splits=5, shuffle=True, random_state=42)
for (ix_train, ix_test) in kfs.split(X_example):
    print("train ix:", ix_train, "test ix:", ix_test)

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
print(f"Size X: {len(X)}, with X train: {len(X_train)} and X test: {len(X_test)}")

In [ ]:
ml_models = {'lm': LinearRegression(),
             'lm_deg2': Pipeline([('poly_transformer', PolynomialFeatures(degree=2)),
                                  ('lm', LinearRegression())]),
             'lm_deg5': Pipeline([('poly_transformer', PolynomialFeatures(degree=5)),
                                  ('lm', LinearRegression())])}

In [ ]:
#from sklearn.model_selection import StratifiedKFold

In [ ]:
kf_results = []

kfs = KFold(n_splits=10, shuffle=True, random_state=42)
for i_f, (ix_train, ix_test) in enumerate(kfs.split(X_train)):
    # Loop over models
    for mod_name, mod in ml_models.items():
        # Define training and testing folds
        X_training_folds = X_train.iloc[ix_train]
        y_training_folds = y_train.iloc[ix_train]
        X_test_fold = X_train.iloc[ix_test]
        y_test_fold = y_train.iloc[ix_test]
        # Fit the model on the training folds
        mod.fit(X_training_folds, y_training_folds)
        # Test on both the training and testing folds to check for over-/under-fitting
        y_pred_train = mod.predict(X_training_folds)
        y_pred_test = mod.predict(X_test_fold)
        # R2
        kf_results.append({'model': mod_name, 'fold': i_f, 'stage': 'train', 'scorer': 'r2', 
                           'val': r2_score(y_training_folds, y_pred_train)})
        kf_results.append({'model': mod_name, 'fold': i_f, 'stage': 'test', 'scorer': 'r2', 
                           'val': r2_score(y_test_fold, y_pred_test)})
        # MSE
        kf_results.append({'model': mod_name, 'fold': i_f, 'stage': 'train', 'scorer': 'MSE', 
                           'val': -mean_squared_error(y_training_folds, y_pred_train)})
        kf_results.append({'model': mod_name, 'fold': i_f, 'stage': 'test', 'scorer': 'MSE', 
                           'val': -mean_squared_error(y_test_fold, y_pred_test)})
kf_results_df = pd.DataFrame(kf_results)

In [ ]:
for mod_name in ml_models.keys():
    kf_df = kf_results_df.loc[kf_results_df['model'] == mod_name]
    with sns.plotting_context("notebook", font_scale=1.2):
        g = sns.catplot(x="fold", y="val", hue="stage", col="scorer", data=kf_df, 
                        kind="bar", sharey=False, height=3, aspect=1.5)
        g.axes[0,0].set_ylim(-0.2,1)
        #g.axes[0,1].set_ylim(0, 45)
        g.fig.suptitle(mod_name, y=1.05)

In [ ]:
with sns.plotting_context("notebook", font_scale=1.2):
    g = sns.catplot(x="model", y="val", hue="stage", col="scorer", data=kf_results_df, 
                    kind="box", sharey=False, height=3, aspect=1.5)
    g.axes[0, 0].set_ylim(-0.2, 1)
    g.axes[0, 1].set_ylim(-50, 0)
    g.fig.suptitle("Score accross fold across models", y=1.05)
    g = sns.catplot(x="model", y="val", hue="stage", col="scorer", data=kf_results_df, 
                    kind="box", sharey=False, height=3, aspect=1.5)
    g.axes[0, 0].set_ylim(0.45, 1)
    g.axes[0, 1].set_ylim(-18, 0)
    g.fig.suptitle("Close up on polynomial models", y=1.05)

#### `sklearn` helps to automate common operations 

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cross_val_score?

In [ ]:
from sklearn.metrics import fbeta_score, make_scorer
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

In [ ]:
ml_models = {'lm': LinearRegression(),
             'lm_deg2': Pipeline([('poly_transformer', PolynomialFeatures(degree=2)),
                                  ('lm', LinearRegression())]),
             'lm_deg5': Pipeline([('poly_transformer', PolynomialFeatures(degree=5)),
                                  ('lm', LinearRegression())])}
# Get cv train AND test scores
cv_test_scores = {}
for mod_name in ml_models.keys():
    cv_test_scores[mod_name] = cross_val_score(ml_models[mod_name], X_train, y_train, 
                                               cv=kfs, scoring=mse_scorer, n_jobs=-1)
cv_test_scores_df = pd.DataFrame(cv_test_scores)

In [ ]:
cv_test_scores_df.boxplot()
plt.title('Negative MSE for the three models tested (larger is better)');

##### Want even more with even less code?

In [ ]:
from sklearn.model_selection import cross_validate

`cross_validate` returns train *and* test scores on *several* metrics

In [ ]:
cross_validate?

In [ ]:
ml_models = {'lm': LinearRegression(),
             'lm_deg2': Pipeline([('poly_transformer', PolynomialFeatures(degree=2)),
                                  ('lm', LinearRegression())]),
             'lm_deg5': Pipeline([('poly_transformer', PolynomialFeatures(degree=5)),
                                  ('lm', LinearRegression())])}
# Get cv train AND test scores
cv_scores = {}
for mod_name in ml_models.keys():
    cv_scores[mod_name] = cross_validate(ml_models[mod_name], X_train, y_train, cv=kfs, 
                                         scoring=['r2', 'neg_mean_squared_error'], 
                                         return_train_score=True, n_jobs=-1)

In [ ]:
def crossval_to_df(cv_dict):
    crossval_results = []
    for model in cv_dict.keys():
        for scorer in cv_dict[model].keys():
            if scorer.startswith('train_'):
                score = scorer.replace('train_', '')
                for i_val, val in enumerate(cv_dict[model][scorer]):
                    crossval_results.append({'model': model, 'fold': i_val, 'stage': 'train', 
                                             'scorer': score, 'val': val})
            elif scorer.startswith('test_'):
                score = scorer.replace('test_', '')
                for i_val, val in enumerate(cv_dict[model][scorer]):
                    crossval_results.append({'model': model, 'fold': i_val, 'stage': 'test', 
                                             'scorer': score, 'val': val})
    return pd.DataFrame(crossval_results)

In [ ]:
crossval_df = crossval_to_df(cv_scores)
for mod_name in crossval_df['model'].unique():
    kf_df = crossval_df.loc[crossval_df['model'] == mod_name]
    with sns.plotting_context("notebook", font_scale=1.2):
        g = sns.catplot(x="fold", y="val", hue="stage", col="scorer", data=crossval_df,
                        kind="bar", sharey=False, height=3, aspect=1.5, errorbar=None)
        g.axes[0,0].set_ylim(-0.2,1)
        g.axes[0,1].set_ylim(-45, 0)
        g.fig.suptitle(mod_name, y=1.05)